## **Breast cancer classification with Keras and Deep Learning**

In [1]:
! pip install gdown

In [2]:
! gdown https://drive.google.com/uc?id=1GGFZNrgbugUtcX3Pgu66t28-GXDFNDMv 

Downloading...
From: https://drive.google.com/uc?id=1GGFZNrgbugUtcX3Pgu66t28-GXDFNDMv
To: /content/7415_10564_bundle_archive.zip
3.33GB [00:52, 63.6MB/s]


In [3]:
%%shell

mkdir breast-cancer-classification

cd breast-cancer-classification
mkdir datasets
mkdir datasets/orig

In [4]:
%%shell
mv /content/7415_10564_bundle_archive.zip /content/breast-cancer-classification/datasets/orig/7415_10564_bundle_archive.zip

In [5]:
%%shell

cd breast-cancer-classification/datasets/orig
unzip 7415_10564_bundle_archive.zip -x "IDC_regular_ps50_idx5/*" 
#ls -al

Streaming output truncated to the last 5000 lines.
  inflating: 9346/0/9346_idx5_x2351_y1601_class0.png  
  inflating: 9346/0/9346_idx5_x2351_y1651_class0.png  
  inflating: 9346/0/9346_idx5_x2351_y1701_class0.png  
  inflating: 9346/0/9346_idx5_x2351_y1751_class0.png  
  inflating: 9346/0/9346_idx5_x2351_y1801_class0.png  
  inflating: 9346/0/9346_idx5_x2351_y1851_class0.png  
  inflating: 9346/0/9346_idx5_x2351_y1901_class0.png  
  inflating: 9346/0/9346_idx5_x2351_y1951_class0.png  
  inflating: 9346/0/9346_idx5_x2351_y2001_class0.png  
  inflating: 9346/0/9346_idx5_x2351_y2251_class0.png  
  inflating: 9346/0/9346_idx5_x2351_y2301_class0.png  
  inflating: 9346/0/9346_idx5_x2351_y2351_class0.png  
  inflating: 9346/0/9346_idx5_x2351_y2401_class0.png  
  inflating: 9346/0/9346_idx5_x2401_y1101_class0.png  
  inflating: 9346/0/9346_idx5_x2401_y1151_class0.png  
  inflating: 9346/0/9346_idx5_x2401_y1201_class0.png  
  inflating: 9346/0/9346_idx5_x2401_y1251_class0.png  
  inflating: 9

In [6]:
# python config.py

# import the necessary packages
import os

# initialize the path to the *original* input directory of images
ORIG_INPUT_DATASET = "/content/breast-cancer-classification/datasets/orig"

# initialize the base path to the *new* directory that will contain
# our images after computing the training and testing split
BASE_PATH = "/content/breast-cancer-classification/datasets/idc"

# derive the training, validation, and testing directories
TRAIN_PATH = os.path.sep.join([BASE_PATH, "training"])
VAL_PATH = os.path.sep.join([BASE_PATH, "validation"])
TEST_PATH = os.path.sep.join([BASE_PATH, "testing"])

# define the amount of data that will be used training
TRAIN_SPLIT = 0.8

# the amount of validation data will be a percentage of the
# *training* data
VAL_SPLIT = 0.1

In [7]:
# python build_dataset.py

# import the necessary packages

#from pyimagesearch import config modified
from imutils import paths
import random
import shutil
import os

# grab the paths to all input images in the original input directory
# and shuffle them
#imagePaths = list(paths.list_images(config.ORIG_INPUT_DATASET)) modified
imagePaths = list(paths.list_images(ORIG_INPUT_DATASET))
random.seed(42)
random.shuffle(imagePaths)

# compute the training and testing split
#i = int(len(imagePaths) * config.TRAIN_SPLIT) modified
i = int(len(imagePaths) * TRAIN_SPLIT)
trainPaths = imagePaths[:i]
testPaths = imagePaths[i:]

# we'll be using part of the training data for validation
#i = int(len(trainPaths) * config.VAL_SPLIT) modified
i = int(len(trainPaths) * VAL_SPLIT)
valPaths = trainPaths[:i]
trainPaths = trainPaths[i:]

# define the datasets that we'll be building
#
#datasets = [
#	("training", trainPaths, config.TRAIN_PATH),
#	("validation", valPaths, config.VAL_PATH),
#	("testing", testPaths, config.TEST_PATH)
#] modified

datasets = [
	("training", trainPaths, TRAIN_PATH),
	("validation", valPaths, VAL_PATH),
	("testing", testPaths, TEST_PATH)
]

# loop over the datasets
for (dType, imagePaths, baseOutput) in datasets:
	# show which data split we are creating
	print("[INFO] building '{}' split".format(dType))

	# if the output base output directory does not exist, create it
	if not os.path.exists(baseOutput):
		print("[INFO] 'creating {}' directory".format(baseOutput))
		os.makedirs(baseOutput)

	# loop over the input image paths
	for inputPath in imagePaths:
		# extract the filename of the input image and extract the
		# class label ("0" for "negative" and "1" for "positive")
		filename = inputPath.split(os.path.sep)[-1]
		label = filename[-5:-4]

		# build the path to the label directory
		labelPath = os.path.sep.join([baseOutput, label])

		# if the label output directory does not exist, create it
		if not os.path.exists(labelPath):
			print("[INFO] 'creating {}' directory".format(labelPath))
			os.makedirs(labelPath)

		# construct the path to the destination image and then copy
		# the image itself
		p = os.path.sep.join([labelPath, filename])
		shutil.copy2(inputPath, p)
		

[INFO] building 'training' split
[INFO] 'creating /content/breast-cancer-classification/datasets/idc/training' directory
[INFO] 'creating /content/breast-cancer-classification/datasets/idc/training/1' directory
[INFO] 'creating /content/breast-cancer-classification/datasets/idc/training/0' directory
[INFO] building 'validation' split
[INFO] 'creating /content/breast-cancer-classification/datasets/idc/validation' directory
[INFO] 'creating /content/breast-cancer-classification/datasets/idc/validation/0' directory
[INFO] 'creating /content/breast-cancer-classification/datasets/idc/validation/1' directory
[INFO] building 'testing' split
[INFO] 'creating /content/breast-cancer-classification/datasets/idc/testing' directory
[INFO] 'creating /content/breast-cancer-classification/datasets/idc/testing/0' directory
[INFO] 'creating /content/breast-cancer-classification/datasets/idc/testing/1' directory


In [8]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

class CancerNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# CONV => RELU => POOL
		model.add(SeparableConv2D(32, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# (CONV => RELU => POOL) * 2
		model.add(SeparableConv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(SeparableConv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# (CONV => RELU => POOL) * 3
		model.add(SeparableConv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(SeparableConv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(SeparableConv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(256))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

In [9]:
# python train_model.py

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
#from pyimagesearch.cancernet import CancerNet
#from pyimagesearch import config
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

# construct the argument parser and parse the arguments : modified at 2020.08.10
#ap = argparse.ArgumentParser()
#ap.add_argument("-p", "--plot", type=str, default="plot.png",
#	help="path to output loss/accuracy plot")
#args = vars(ap.parse_args())

import easydict
 
args = easydict.EasyDict({
		"plot": "plot.png"
})

# initialize our number of epochs, initial learning rate, and batch
# size
NUM_EPOCHS = 40
INIT_LR = 1e-2
BS = 32

# determine the total number of image paths in training, validation,
# and testing directories

#trainPaths = list(paths.list_images(config.TRAIN_PATH)) modified
#totalTrain = len(trainPaths)
#totalVal = len(list(paths.list_images(config.VAL_PATH))) modified
#totalTest = len(list(paths.list_images(config.TEST_PATH))) modified

trainPaths = list(paths.list_images(TRAIN_PATH))
totalTrain = len(trainPaths)
totalVal = len(list(paths.list_images(VAL_PATH)))
totalTest = len(list(paths.list_images(TEST_PATH)))

# calculate the total number of training images in each class and
# initialize a dictionary to store the class weights
trainLabels = [int(p.split(os.path.sep)[-2]) for p in trainPaths]
trainLabels = to_categorical(trainLabels)
classTotals = trainLabels.sum(axis=0)
classWeight = dict()

# loop over all classes and calculate the class weight
for i in range(0, len(classTotals)):
	classWeight[i] = classTotals.max() / classTotals[i]

# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rescale=1 / 255.0,
	rotation_range=20,
	zoom_range=0.05,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.05,
	horizontal_flip=True,
	vertical_flip=True,
	fill_mode="nearest")

# initialize the validation (and testing) data augmentation object
valAug = ImageDataGenerator(rescale=1 / 255.0)

# initialize the training generator
trainGen = trainAug.flow_from_directory(
	TRAIN_PATH,
	class_mode="categorical",
	target_size=(48, 48),
	color_mode="rgb",
	shuffle=True,
	batch_size=BS)

# initialize the validation generator
valGen = valAug.flow_from_directory(
	VAL_PATH,
	class_mode="categorical",
	target_size=(48, 48),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)

# initialize the testing generator
testGen = valAug.flow_from_directory(
	TEST_PATH,
	class_mode="categorical",
	target_size=(48, 48),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)

# initialize our CancerNet model and compile it
model = CancerNet.build(width=48, height=48, depth=3,
	classes=2)
opt = Adagrad(lr=INIT_LR, decay=INIT_LR / NUM_EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# fit the model
H = model.fit(
	x=trainGen,
	steps_per_epoch=totalTrain // BS,
	validation_data=valGen,
	validation_steps=totalVal // BS,
	class_weight=classWeight,
	epochs=NUM_EPOCHS)

# reset the testing generator and then use our trained model to
# make predictions on the data
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict(x=testGen, steps=(totalTest // BS) + 1)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))

# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity
cm = confusion_matrix(testGen.classes, predIdxs)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])

# show the confusion matrix, accuracy, sensitivity, and specificity
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))

# plot the training loss and accuracy
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

Found 199818 images belonging to 2 classes.
Found 22201 images belonging to 2 classes.
Found 55505 images belonging to 2 classes.
Epoch 1/40
6244/6244 [==============================] - 255s 41ms/step - loss: 0.6233 - accuracy: 0.8151 - val_loss: 0.4203 - val_accuracy: 0.8272
Epoch 2/40
6244/6244 [==============================] - 250s 40ms/step - loss: 0.5667 - accuracy: 0.8291 - val_loss: 0.4162 - val_accuracy: 0.8313
Epoch 3/40
6244/6244 [==============================] - 249s 40ms/step - loss: 0.5600 - accuracy: 0.8317 - val_loss: 0.4202 - val_accuracy: 0.8265
Epoch 4/40
6244/6244 [==============================] - 250s 40ms/step - loss: 0.5548 - accuracy: 0.8335 - val_loss: 0.4236 - val_accuracy: 0.8255
Epoch 5/40
6244/6244 [==============================] - 249s 40ms/step - loss: 0.5514 - accuracy: 0.8339 - val_loss: 0.4107 - val_accuracy: 0.8312
Epoch 6/40
6244/6244 [==============================] - 249s 40ms/step - loss: 0.5499 - accuracy: 0.8350 - val_loss: 0.4051 - val_accur